# Basic stats on the evolution of COVID-19 hospitalizations in departments

In [1]:
%load_ext lab_black
%matplotlib inline

In [2]:
import matplotlib.pyplot as plt
from datetime import timedelta, date
import pandas as pd
import warnings

warnings.filterwarnings("ignore")
pd.plotting.register_matplotlib_converters()

In [3]:
covid_url = (
    "https://www.data.gouv.fr/fr/datasets/r/63352e38-d353-4b54-bfd1-f1b3ee1cabd7"
)
filter_dep = ["971", "972", "973", "974", "976"]  # only metropolitan France
figsize = (15, 10)

#### <br> Load table with COVID-19 data (figures by dates)

In [4]:
df = pd.read_csv(covid_url, sep=";", parse_dates=True, index_col=2)
df = df.query("sexe == 0")  # sum male/female
df.drop(columns=["sexe"], inplace=True)
df.query("dep not in @filter_dep", inplace=True)
df.sort_index(inplace=True)
df.head(2)

,dep,hosp,rea,rad,dc
jour,,,,,
2020-03-18,01,2,0,1,0
2020-03-18,02,41,10,18,11


In [5]:
last_day = df.index.max()
previous_day = last_day - timedelta(days=1)
print("Evolution from ", previous_day, " to ", last_day)

Evolution from  2020-04-19 00:00:00  to  2020-04-20 00:00:00


#### <br> Build a new dataframe with data from two latest days and the associated evolution

In [6]:
sdf = df.loc[last_day]
sdf.reset_index(inplace=True)
pdf = df.loc[previous_day]
pdf.reset_index(inplace=True)
frame = {"dep": sdf["dep"], "prev_hosp": pdf["hosp"], "last_hosp": sdf["hosp"]}
stats = pd.DataFrame(frame)
stats["diff"] = stats["last_hosp"] - stats["prev_hosp"]
stats["percent"] = stats["diff"] / stats["prev_hosp"] * 100
stats.head()

,dep,prev_hosp,last_hosp,diff,percent
0,01,172,170,-2,-1.162791
1,02,275,304,29,10.545455
2,03,57,62,5,8.771930
3,04,38,45,7,18.421053
4,05,38,38,0,0.000000


#### <br> Departments with highest number of hospitalizations on the latest day

In [7]:
stats.sort_values(by="last_hosp", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
75,75,2939,2933,-6,-0.204151
94,94,2213,2215,2,0.090375
92,92,1982,1983,1,0.050454
93,93,1568,1562,-6,-0.382653
69,69,1243,1229,-14,-1.126307


#### <br> Min, max, mean of evolution between the two latest days (in sum and %)

In [8]:
print(
    "Min: ",
    stats["diff"].min(),
    " Mean: ",
    stats["diff"].mean(),
    " Max: ",
    stats["diff"].max(),
)

Min:  -23  Mean:  -0.14583333333333334  Max:  30


In [9]:
print(
    "Min %: ",
    stats.percent.min(),
    " Mean %: ",
    stats.percent.mean(),
    " Max %: ",
    stats.percent.max(),
)

Min %:  -27.27272727272727  Mean %:  -0.1831348894399527  Max %:  18.421052631578945


#### <br> List of departments with a drecrease of hospitalizations (last two days)

In [10]:
stats.query("diff < 0")

,dep,prev_hosp,last_hosp,diff,percent
0,01,172,170,-2,-1.162791
5,06,279,276,-3,-1.075269
6,07,111,103,-8,-7.207207
7,08,67,66,-1,-1.492537
9,10,268,256,-12,-4.477612
10,11,70,65,-5,-7.142857
13,14,145,144,-1,-0.689655
15,16,13,12,-1,-7.692308
16,17,84,83,-1,-1.190476
20,22,65,63,-2,-3.076923


In [11]:
print(stats.query("diff < 0").shape)

(48, 5)


#### <br> Departments with highest evolution of hospitalizations (total number)

In [12]:
stats.sort_values(by="diff", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
59,59,927,957,30,3.236246
1,02,275,304,29,10.545455
68,68,957,976,19,1.985371
30,30,118,136,18,15.254237
95,95,910,927,17,1.868132


#### <br> Departments with highest rate of hospitalizations (%)

In [13]:
stats.sort_values(by="percent", ascending=False).head()

,dep,prev_hosp,last_hosp,diff,percent
3,04,38,45,7,18.421053
86,86,51,59,8,15.686275
32,32,45,52,7,15.555556
30,30,118,136,18,15.254237
22,24,54,61,7,12.962963


#### <br> Focus on some departments

In [14]:
stats.query("dep=='75'")

,dep,prev_hosp,last_hosp,diff,percent
75,75,2939,2933,-6,-0.204151


In [15]:
stats.query("dep=='71'")

,dep,prev_hosp,last_hosp,diff,percent
71,71,202,209,7,3.465347


In [16]:
stats.query("dep=='31'")

,dep,prev_hosp,last_hosp,diff,percent
31,31,195,199,4,2.051282
